<span style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">An Exception was encountered at '<a href="#papermill-error-cell">In [8]</a>'.</span>

In [1]:
import pandas as pd
import numpy as np
import pyodbc 
import time
from datetime import datetime
from datetime import date
import sqlite3

In [2]:
import logging
path = "..\..\InSyncConnection\Code\clinical_log.txt"
logging.basicConfig(filename=path,
                    filemode='a',
                    format='%(asctime)s,%(msecs)d,%(name)s,%(levelname)s,%(message)s',
                    datefmt='%Y-%m-%d %H:%M:%S',
                    level=logging.DEBUG)
logger = logging.getLogger("Determine Discharge Status")

In [3]:
def findPatientsNotSeenInaWhile(lastApptDate):
    try:
        if (datetime.now() - lastApptDate).days > 91:
            if (datetime.now() - lastApptDate).days > 183:
                return "Hasn't been seen in 6 months" 
            else:
                return "Hasn't been seen in 3 months"
        else:
            return "They've been seen in the past 3 months" 
    except:
        
        return "Never Seen"   

In [4]:
# diff path from TestTev vs WorkingEnv
conn = sqlite3.connect('../../InSyncConnection/Database/InSyncClinical.db')
# cursor = conn.cursor()

# printing all table names  
# sql_query = """SELECT name FROM sqlite_master
#     WHERE type='table';"""

# cursor.execute(sql_query)
# print(cursor.fetchall())

# Read in Data

In [5]:
testPatients = '''
SELECT
    MRNNumber
FROM 
    emr_PatientDetails
WHERE 
    LOWER(FirstName) LIKE '%test%'
    OR LOWER(FirstName) LIKE '%patient%'
    OR LOWER(FirstName) LIKE '%new%'
    OR LOWER(FirstName) LIKE '%client%'
    OR LOWER(FirstName) LIKE '%release%'
    OR LOWER(LastName) LIKE '%test%'
    OR LOWER(LastName) LIKE '%patient%'
    OR LOWER(LastName) LIKE '%fake%'
    OR CAST(MRNNumber AS INTEGER) < 53
    OR CAST(MRNNumber AS INTEGER) = 54
    OR CAST(MRNNumber AS INTEGER) = 141
'''

In [6]:
sql=f'''
SELECT 
    tblEncounterSummary.PatientID as PatientID,
    emr_PatientDetails.isActive,
    tblEncounterSummary.VisitDateTime as EncounterDate,
    tblEncounterType.EncounterType
FROM 
    tblEncounterSummary
    LEFT JOIN emr_PatientDetails ON (tblEncounterSummary.patientID = emr_PatientDetails.patientID)
    LEFT JOIN tblEncounterType ON (tblEncounterSummary.EncounterTypeID=tblEncounterType.EncounterTypeID)
WHERE
    emr_PatientDetails.MRNNumber NOT IN ({testPatients})
'''
try:
    encounters = pd.read_sql(sql, conn)
    logger.info("Successfully queried encounters")
except Exception as e:
    logger.error(f"Failed to query encounters.", exc_info=True) 
    print(e)

In [7]:
sql=f'''
SELECT 
    tblEncounterSummary.PatientID as PatientID,
    tblEncounterSummary.VisitDateTime as Billable_EncounterDate,
    tblEncounterType.EncounterType as Billable_EncounterType,
    tblEncounterSummary.ProgramName,
    tblFacilitiesMaster.FacilityName
FROM 
    tblEncounterSummary
    LEFT JOIN emr_PatientDetails ON (tblEncounterSummary.patientID = emr_PatientDetails.patientID)
    LEFT JOIN tblEncounterType ON (tblEncounterSummary.EncounterTypeID=tblEncounterType.EncounterTypeID)
    LEFT JOIN tblFacilitiesMaster ON (tblEncounterSummary.FacilityId=tblFacilitiesMaster.FacilityId)
WHERE IsBillable LIKE "%TRUE%" 
    AND emr_PatientDetails.MRNNumber NOT IN ({testPatients})
'''
try:
    billableEncounters = pd.read_sql(sql, conn)
    logger.info("Successfully queried billableEncounters = pd.read_sql(sql, conn)")
except Exception as e:
    logger.error(f"Failed to query billableEncounters = pd.read_sql(sql, conn).", exc_info=True) 
    print(e)


# Transform Data

<span id="papermill-error-cell" style="color:red; font-family:Helvetica Neue, Helvetica, Arial, sans-serif; font-size:2em;">Execution using papermill encountered an exception here and stopped:</span>

In [8]:
# Get last ecounter
encounters['EncounterDate'] = pd.to_datetime(encounters['EncounterDate'])
encounters = encounters.sort_values(by="EncounterDate")
# get last billable encounter
billableEncounters['Billable_EncounterDate'] = pd.to_datetime(billableEncounters['Billable_EncounterDate'])
billableEncounters = billableEncounters.sort_values(by="Billable_EncounterDate")

ValueError: time data "8/10/2022" doesn't match format "%m/%d/%Y %I:%M:%S %p", at position 136039. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.

In [ ]:
# get the most recent (last) encounter type, date and patient activity
discharged = encounters.groupby(['PatientID'], as_index=False)[['EncounterType', 'EncounterDate', 'IsActive']].last()
finalBillable = billableEncounters.groupby(['PatientID'], as_index=False)[['Billable_EncounterType', 'Billable_EncounterDate']].last()
finalEncounterFacility = billableEncounters.groupby(['PatientID'], as_index=False)[['ProgramName', 'FacilityName']].last()

In [ ]:
# check if encounter type contains discharge
discharged['is_Discharged'] = discharged['EncounterType'].map(lambda type: 1 if "discharge" in str(type).lower() else 0)
# summarize last seen date
finalBillable['seen_Recently'] = finalBillable['Billable_EncounterDate'].map(findPatientsNotSeenInaWhile)

In [ ]:
finalPatientStatus = discharged.merge(finalBillable, how='left', on='PatientID').merge(finalEncounterFacility, how='left', on='PatientID')

In [ ]:
# rename columns to be more explicit
finalPatientStatus.columns = ['PatientID', 
                      'finalEncounterType', 
                      'finalEncounterDate', 
                      'is_ActiveInInSync',
                      'is_lastEncounterDischarge',
                      'lastBillable_EncounterType',
                      'lastBillable_EncounterDate',
                      'seen_Recently',
                      'lastBillable_ProgramName',
                      'lastBillable_FacilityName']
#reorder the columns
finalPatientStatus =    finalPatientStatus[['PatientID', 
                      'finalEncounterType', 
                      'finalEncounterDate', 
                      'is_ActiveInInSync',
                      'is_lastEncounterDischarge',
                      'lastBillable_EncounterType',
                      'lastBillable_EncounterDate',
                      'lastBillable_ProgramName',
                      'lastBillable_FacilityName',
                      'seen_Recently']       ]

In [ ]:
finalPatientStatus[finalPatientStatus['PatientID'] == 622719]

## Manual Line
- The code below is used to help with writing the proper SQL Create table query below

In [ ]:
# use to transform data into
", ".join([item + " " + str(finalPatientStatus[item].dtype) for item in finalPatientStatus.columns])

# Push Data to Table

In [ ]:
table_name= "ptPatient_Activity"
try:
    c = conn.cursor()
    c.execute(f'''CREATE TABLE IF NOT EXISTS {table_name} (PatientID INTEGER, finalEncounterType TEXT, finalEncounterDate TEXT, is_ActiveInInSync INTEGER, is_lastEncounterDischarge INTEGER, lastBillable_EncounterType TEXT, lastBillable_EncounterDate TEXT, lastBillable_ProgramName TEXT, lastBillable_FacilityName TEXT, seen_Recently TEXT) ''')
    conn.commit()
    finalPatientStatus.to_sql("ptPatient_Activity", conn, if_exists='replace', index = False)
    logger.info(f"Successfully pushed {table_name} to database.")            
except Exception as e:
    logger.error(f"Failed to push {table_name} to database.", exc_info=True) 
    print(e)

In [ ]:
conn.close()